In [2]:
#Imports

#re
import re

#Import Pandas and Numpy
import numpy as np
import pandas as pd

#Web crawler
from crawler import crawler

#nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize, sent_tokenize
from nltk.stem import WordNetLemmatizer 

#Gensim
import gensim
import gensim.corpora as corpora
import gensim.models.ldamodel as lda
from gensim.models import CoherenceModel

# Plotting tools
import pyLDAvis
import pyLDAvis.gensim
import matplotlib.pyplot as plt


In [9]:
#################
#DATA RETRIEVAL

#Get articles in

#Create crawler object
web_crawl = crawler()

# Define Google Search query and number of searches
google_query = ['site:longreads.com america -tag']
num_search = 1

#Call the crawler
#Store the df that is returned in a pandas dataframe
data = web_crawl.write_crawl_results(google_query, num_search)

#1st article    
text = data["basic"][0]

#Col 1 - basic - Cleaned, but otherwise untouched body of text
#Col 2 - label - Query for the search query


In [47]:
#################
#PRE-PROCESSING - Article

def pre_processing(article):
    #Force everything to lower case
    article_lower = article.lower()
    
    ##Remove non-words
    article_clean = re.sub(r"[^\w\s\d]+", " ", article_lower)
    
    #Tokenise the article
    article_tokenise = word_tokenize(article_clean)
    
    ##Remove stopwords
    #Create a stopwords object
    stop_words = set(stopwords.words("english"))
    print(stop_words)
    
    #Extending the stopwords list with "u" - somehow this even showed up in the LDA
    #I think it comes from U.S. being separated to "u" and "s"
    #Longreads and wordpress are the names of the website and blog that put out the article and
    #add no meaning
    stop_words.add('world') #Why didn't this get rid of the u?
    
    #Keep only words not in the stopwords list
    article_sans_stopwords = [each_word for each_word in article_tokenise if each_word not in stop_words]
    
    ##Lemmatise
    #Create lemmatise object
    lemmatiser = WordNetLemmatizer()
    
    #Lemmatise~~ - should try to deal with this keeping only nouns, adjectives, verbs, and adverbs?
    article_lemmatised = [lemmatiser.lemmatize(word) for word in article_sans_stopwords]
    
    #Return lemmatised article
    #return article_lower, article_clean, article_tokenise, article_sans_stopwords, article_lemmatised
    return article_lemmatised

{"didn't", 'here', 'myself', "won't", 'very', 'has', "needn't", 'why', 'be', 's', 'll', "hasn't", 'above', 'off', 'me', 'yours', 'am', "don't", 'the', 'you', "couldn't", "wasn't", 'at', 'those', 'mightn', 'when', 'and', 're', 've', 'weren', 'needn', 'once', 'hasn', 'her', 'mustn', 'of', 'between', 'below', 'not', 'just', 'other', 'where', 'themselves', 'them', 'all', "you'll", "shan't", "doesn't", 'doesn', 'he', 'ain', 'further', 'do', "wouldn't", 'after', 'don', 'shan', 'but', 'yourself', 'having', 'isn', 'to', 'have', 'down', "you've", 'they', 'hadn', 'my', 'ours', 'this', 'during', 'what', 'in', 'out', 'hers', 'up', 'same', "hadn't", "aren't", 'ourselves', 'didn', 'i', 'been', 'through', 'so', 'their', 'is', "shouldn't", 'on', 'theirs', 'couldn', 'that', 'should', 'any', 'himself', 'now', 'she', 'm', 'only', 'yourselves', 'whom', 'haven', 'most', 'shouldn', 'herself', 'doing', 'under', 'how', 'd', 'few', 't', 'ma', 'your', 'did', 'can', "mightn't", "mustn't", 'because', "should've",

['x']

In [48]:
#################
#PRE-PROCESSING - Sentences
def sentence_processing(article):
    #Sentence tokenisation
    sentences = sent_tokenize(article)
    
    #Get clean, lemmatised sentences without stopwords
    sentences_lemmatised = [pre_processing(sentence) for sentence in sentences]
    
    #Return lemmatised sentence list
    return sentences_lemmatised

In [49]:
#################
#COUNTING - Sentences

def sent_tfidf(article_lemmatised_sentences):
    #Import sklearn
    from sklearn.feature_extraction.text import TfidfVectorizer
    
    # Create the TFIDF Object
    tfidf_vectoriser = TfidfVectorizer()
    
    #Join sentences for the vectoriser
    sentence_list = [" ".join(sentence) for sentence in article_lemmatised_sentences]
    
    # Fit sentences to the TFIDF vectoriser
    senteces_TFIDFed = tfidf_vectoriser.fit_transform(sentence_list)
    
    # Put the TFIDF vectorised data into a pandas dataframe
    sent_df = pd.DataFrame(senteces_TFIDFed.toarray())
    
    # Rename the columns to the words in the sentences
    sent_df.columns = tfidf_vectoriser.get_feature_names()
    
    return sent_df

In [13]:
#TFIDF our sentences to see which has the important words
#tfidf_article_by_sent = sent_tfidf(sent_lemmatised)

In [50]:
#Lemmatise sentences
sent_lemmatised = sentence_processing(text)

{"didn't", 'here', 'myself', "won't", 'very', 'has', "needn't", 'why', 'be', 's', 'll', "hasn't", 'above', 'off', 'me', 'yours', 'am', "don't", 'the', 'you', "couldn't", "wasn't", 'at', 'those', 'mightn', 'when', 'and', 're', 've', 'weren', 'needn', 'once', 'hasn', 'her', 'mustn', 'of', 'between', 'below', 'not', 'just', 'other', 'where', 'themselves', 'them', 'all', "you'll", "shan't", "doesn't", 'doesn', 'he', 'ain', 'further', 'do', "wouldn't", 'after', 'don', 'shan', 'but', 'yourself', 'having', 'isn', 'to', 'have', 'down', "you've", 'they', 'hadn', 'my', 'ours', 'this', 'during', 'what', 'in', 'out', 'hers', 'up', 'same', "hadn't", "aren't", 'ourselves', 'didn', 'i', 'been', 'through', 'so', 'their', 'is', "shouldn't", 'on', 'theirs', 'couldn', 'that', 'should', 'any', 'himself', 'now', 'she', 'm', 'only', 'yourselves', 'whom', 'haven', 'most', 'shouldn', 'herself', 'doing', 'under', 'how', 'd', 'few', 't', 'ma', 'your', 'did', 'can', "mightn't", "mustn't", 'because', "should've",

In [51]:
#################
#LDA

#Create LDA dictionary and corpus
dictionary = corpora.Dictionary(sent_lemmatised)
corpus = [dictionary.doc2bow(text) for text in sent_lemmatised]
#Btw the lists are still sentences, and the tuples are word id, and the count (within the sentence)


#Create model
lda_model = lda.LdaModel(corpus,
                         id2word = dictionary,
                         num_topics = 6
                         )
topics = lda_model.print_topics(num_words = 10)

for topics in topics:
    print(topics)


(0, '0.031*"fay" + 0.027*"vincent" + 0.016*"wright" + 0.011*"though" + 0.010*"find" + 0.008*"america" + 0.007*"life" + 0.006*"hard" + 0.006*"mojave" + 0.006*"protest"')
(1, '0.031*"vincent" + 0.020*"fay" + 0.012*"like" + 0.011*"wright" + 0.009*"one" + 0.007*"never" + 0.006*"house" + 0.006*"become" + 0.006*"shelter" + 0.005*"come"')
(2, '0.015*"fay" + 0.009*"space" + 0.008*"love" + 0.008*"get" + 0.008*"nasa" + 0.008*"like" + 0.007*"start" + 0.006*"vincent" + 0.005*"find" + 0.005*"reader"')
(3, '0.019*"wright" + 0.017*"fay" + 0.014*"alcott" + 0.010*"one" + 0.009*"first" + 0.009*"vincent" + 0.008*"novel" + 0.008*"time" + 0.007*"never" + 0.007*"father"')
(4, '0.012*"fay" + 0.011*"wright" + 0.009*"vincent" + 0.008*"moon" + 0.006*"mother" + 0.006*"right" + 0.006*"mourning" + 0.006*"two" + 0.006*"across" + 0.006*"ever"')
(5, '0.018*"alcott" + 0.012*"change" + 0.012*"berrigan" + 0.008*"america" + 0.007*"life" + 0.007*"fay" + 0.007*"like" + 0.007*"way" + 0.006*"love" + 0.006*"question"')


In [52]:
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(lda_model, corpus, dictionary, sort_topics = False)
vis

C:\ProgramData\Anaconda3\lib\site-packages\pyLDAvis\_prepare.py:257: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  return pd.concat([default_term_info] + list(topic_dfs))


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
0      0.096074 -0.057987       1        1  20.734814
1      0.065327  0.036187       2        1  19.860977
2     -0.011220 -0.036705       3        1  11.627600
3     -0.032965  0.109016       4        1  17.882286
4     -0.008700  0.009095       5        1  10.075588
5     -0.108515 -0.059606       6        1  19.818739, topic_info=    Category       Freq       Term      Total  loglift  logprob
2    Default  12.000000     alcott  12.000000  30.0000  30.0000
138  Default  18.000000     wright  18.000000  29.0000  29.0000
639  Default   4.000000     change   4.000000  28.0000  28.0000
134  Default  29.000000    vincent  29.000000  27.0000  27.0000
261  Default   5.000000     though   5.000000  26.0000  26.0000
38   Default   8.000000        one   8.000000  25.0000  25.0000
124  Default   5.000000      first   5.000000  24.0000  24.0000
388  Default   3.000000    shelter   3.000000  23.0000  23.0000
506  Default   2.000000       nasa   2.000000  22.0000  22.0000
13   Default   6.000000      novel   6.000000  21.0000  21.0000
105  Default   3.000000        two   3.000000  20.0000  20.0000
374  Default   5.000000        get   5.000000  19.0000  19.0000
108  Default  33.000000        fay  33.000000  18.0000  18.0000
4    Default   9.000000   berrigan   9.000000  17.0000  17.0000
102  Default   5.000000       moon   5.000000  16.0000  16.0000
702  Default   3.000000     mother   3.000000  15.0000  15.0000
192  Default   6.000000       love   6.000000  14.0000  14.0000
354  Default   4.000000     reader   4.000000  13.0000  13.0000
839  Default   2.000000       ever   2.000000  12.0000  12.0000
843  Default   2.000000      right   2.000000  11.0000  11.0000
60   Default   9.000000       find   9.000000  10.0000  10.0000
210  Default   3.000000      point   3.000000   9.0000   9.0000
62   Default   6.000000       hard   6.000000   8.0000   8.0000
310  Default   4.000000    instead   4.000000   7.0000   7.0000
841  Default   1.000000   mourning   1.000000   6.0000   6.0000
97   Default   1.000000     across   1.000000   5.0000   5.0000
699  Default   1.000000   anything   1.000000   4.0000   4.0000
117  Default   6.000000      space   6.000000   3.0000   3.0000
637  Default   2.000000     public   2.000000   2.0000   2.0000
811  Default   3.000000     braden   3.000000   1.0000   1.0000
..       ...        ...        ...        ...      ...      ...
85    Topic6   1.268844  contained   2.061250   1.1333  -5.6695
285   Topic6   1.257544   clothing   2.050908   1.1294  -5.6785
2     Topic6   6.766013     alcott  12.983958   0.9667  -3.9957
12    Topic6   1.204173  manifesto   2.006358   1.1080  -5.7218
6     Topic6   1.183413    dilemma   1.988387   1.0996  -5.7392
80    Topic6   1.182458     spirit   1.992806   1.0966  -5.7400
692   Topic6   1.182021       idea   1.995529   1.0949  -5.7404
4     Topic6   4.280775   berrigan   9.319736   0.8405  -4.4535
280   Topic6   2.477702        way   5.462495   0.8280  -5.0003
94    Topic6   1.680773        set   3.399859   0.9141  -5.3884
404   Topic6   1.392870    present   2.729494   0.9458  -5.5763
149   Topic6   2.241190   question   5.353500   0.7478  -5.1006
484   Topic6   1.542006       full   3.262968   0.8690  -5.4746
21    Topic6   1.560948   american   3.391088   0.8427  -5.4623
51    Topic6   2.879757    america   8.539312   0.5316  -4.8499
340   Topic6   1.519451      could   3.282084   0.8484  -5.4893
75    Topic6   1.561157       book   3.428383   0.8319  -5.4622
192   Topic6   2.281621       love   6.272856   0.6072  -5.0828
65    Topic6   2.085306     letter   5.867612   0.5840  -5.1727
165   Topic6   2.735151       life   9.945628   0.3276  -4.9014
62    Topic6   2.087364       hard   6.462873   0.4884  -5.1717
345   Topic6   2.577965       like  11.817448   0.0960  -4.9606
60    Topic6   2.192135       find   9.082765   0.1970  -5.1228
13   

In [54]:
# Topic coherence
# Read the explanations from https://datascienceplus.com/evaluation-of-topic-modeling-topic-coherence/
# and https://rare-technologies.com/what-is-topic-coherence/
#But don't completely get it. What's important though, is that topic coherence gives us a quantitative measure
# of topic extraction quality from our LDA model.


#THIS DOESN'T WORK FOR SOME REASON ):
coherence_model_lda = CoherenceModel(model = lda_model,
                                     texts = sent_lemmatised,
                                     dictionary = dictionary,
                                     coherence='c_v'
                                     )
coherence_lda = coherence_model_lda.get_coherence()

In [55]:
print(coherence_lda)

0.46806453717447005
